In [ ]:
import requests
import pandas as pd
import

json_url = 'http://localhost:1111/manutencao'
try:

    response = requests.get(json_url)
    response.raise_for_status() 
    from io import StringIO
    df = pd.read_json(StringIO(response.text))
    print("DataFrame criado com sucesso!")

except requests.exceptions.ConnectionError:
    print("ERRO: Falha na conexão. Servidor recusou a conexão (Connection Refused).")

except requests.exceptions.HTTPError as err:
    print(f"ERRO HTTP: A requisição falhou com o status: {err.response.status_code}")

except Exception as e:
    print(f"Ocorreu um erro geral: {e}")
df.head(10)

DataFrame criado com sucesso!


,id,item,tempo_manutencao,data_manutencao,setor,observacao,responsavel,tipo
0,1,Servidor Principal,90,2025-11-10,Hardware,Substituição de módulos de memória RAM.,João Victor,Preventiva
1,2,Impressora Laser B,45,2025-11-10,Escritorio,Troca de fusor e limpeza interna.,João Victor,Corretiva
2,3,Estação de Trabalho 15,20,2025-11-11,Hardware,Otimização de sistema operacional e drivers.,Anna Karolina,Preditiva
3,4,Máquina de Corte CNC,180,2025-11-12,Producao,Calibração dos eixos X e Y. Manutenção prevent...,Breno,Preditiva
4,5,Ar Condicionado Sala A,60,2025-11-12,Infraestrutura,Limpeza e recarga de gás refrigerante.,Breno,Preventiva
5,6,teste,105,2025-11-12,Almoxarifado,concerto,João Victor,Preventiva
6,7,Mesa,105,2025-12-03,RH,concerto,Alice,preventiva
7,8,Cadeira,105,2025-12-03,RH,,Alice,Preventiva
8,9,Cadeira,105,2025-12-03,RH,,João,Preventiva
9,10,Impressora Jato C,10,2025-12-08,Escritorio,Problema original: Papel atolando e ruído alto...,Anna Karolina,Preventiva


In [4]:
df = df[['setor', 'tempo_manutencao', 'responsavel', 'tipo']]

X = df.drop('tempo_manutencao', axis=1)
y = df['tempo_manutencao']

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 2. PREPARAÇÃO DO DATASET PARA TREINAMENTO ---

features_to_drop = ['id', 'data_manutencao', 'tempo_manutencao']
X = df.drop(features_to_drop, axis=1)
y = df['tempo_manutencao']

categorical_features = X.columns.tolist()

# Define o pré-processador: OneHotEncoder para todas as colunas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Separação (Necessária para o método .fit do Pipeline)
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.01, random_state=42)

# --- 3. TREINAMENTO DO MODELO XGBOOST ---

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=100, 
    random_state=42, 
    learning_rate=0.1, 
    max_depth=3
)

# Cria e treina o pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

print("Iniciando treinamento do XGBoost no histórico de manutenção...")
# Treinamento usa o pipeline para aplicar OHE e treinar o modelo
model_pipeline.fit(X_train, y_train)
print("Treinamento concluído! Modelo pronto para previsões.")

# =========================================================================
# --- 4. PREVISÃO DE UMA NOVA MANUTENÇÃO ---
# =========================================================================

# Dados da Nova Solicitação a ser prevista
novo_registro = {
    'item': ['Máquina de Corte CNC'],
    'data_manutencao': ['2026-01-15'], # Sera descartado, mas deve estar presente para consistência
    'setor': ['Producao'],
    'observacao': ['Substituição e calibração de um eixo Y quebrado.'],
    'responsavel': ['Mariana Torres'],
    'tipo': ['Corretiva']
}

# Cria o DataFrame para a previsão
novo_dado_df = pd.DataFrame(novo_registro)

# Remove as colunas não utilizadas no treinamento (id e data_manutencao)
novo_dado_df = novo_dado_df.drop('data_manutencao', axis=1, errors='ignore')

# Faz a previsão usando o pipeline treinado
# O pipeline garante que o novo dado seja transformado corretamente (OHE) antes da previsão
tempo_previsto = model_pipeline.predict(novo_dado_df)

# Extrai o valor estimado
tempo_estimado = tempo_previsto[0]

# --- 5. RESULTADO DA PREVISÃO ---

print("\n--- Resultado da Previsão ---")
print(f"Detalhes da Manutenção:")
print(f"  Item: {novo_registro['item'][0]}")
print(f"  Tipo: {novo_registro['tipo'][0]}")
print(f"  Setor: {novo_registro['setor'][0]}")
print(f"  Responsável: {novo_registro['responsavel'][0]}")

print(f"\nO Tempo de Manutenção estimado é de: {tempo_estimado:.2f} minutos.")
print(f"Isso equivale a aproximadamente: {tempo_estimado / 60:.2f} horas.")

ModuleNotFoundError: No module named 'xgboost'